# Python objektově orientované programování

---

1. [Vlastnosti třídy](),
    - [řešení metodou](),
    - [řešení funkcí property](),
    - [getter, setter, deleter](),
2. [polymorfismus](#Polymorfismus-(-~polymorphism)),
    - [terminologie](#Terminologie),
    - [význam v Pythonu](#Význam-v-Pythonu),
    - [využití v OOP](#Využití-v-OOP),
3. [zapouzdření](#Zapouzdření-(~encapsulation)),
    - [bez zapouzdření](#Bez-konceptu-zapouzdření),
    - [privátní objekty](#Privátní-objekty),
    - [gettery, settery](#Gettery,-settery)
    
---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.P14PbLzZfDCpGX8hFnK74wHaHa%26pid%3DApi&f=1&ipt=de07e3110c5b46007e138de2aa1ddf169453c6bdf583d56537c3c749b7cf2055&ipo=images" width="250" style="margin-left:auto; margin-right:auto"/>

## Vlastnosti třídy

---


V některých programovacích jazycích se k atributům **nepřistupuje přímo**.

Je to z toho důvodu, že by uživatel nechtěně pracoval se špatným datovým typem, případně nevalidní hodnotou.

V Pythonu tato funkcionalita **není podporovaná přímo**, ale existuje *workaround*.

<br>

Představ si situaci, kdy máš napsat **převodník jednotek objemu**.

Z **litrů** na **pinty**(UK):

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.FCRrMRa9SQrNjgFSKfKP6AHaHa%26pid%3DApi&f=1&ipt=f45d961e5b9d1d428a1c59180cd22f9c6a298a6a8ccaa753d86ca54f906f045b&ipo=images" width="200" style="margin-left:auto; margin-right:auto"/>

In [1]:
class ObjemovyKonvertor:
    def __init__(self, litr: int):
        self.litr = litr
        self.koeficient_pinta = 1.7598
        self.pinty = self.litr * self.koeficient_pinta

In [2]:
dva_litry = ObjemovyKonvertor(2)

In [3]:
dva_litry.pinty

3.5196

<br>

Jako uživatel ale můžeš zadat **nesprávný argument** pro tvoji třídu:

In [4]:
tri_litry = ObjemovyKonvertor("tři")

TypeError: can't multiply sequence by non-int of type 'float'

<br>

Případně pokud potřebuješ přepočet takové instance, *interpret* nebude na tvoji úpravu reagovat:

In [5]:
dva_litry.litr = 3

In [6]:
dva_litry.pinty

3.5196

In [7]:
dva_litry.litr

3

<br>

*Intepret* v takovém případě pracuje **z první hodnotou**, kterou dostal při *instancování*.

Následná změna má vliv pouze na instanční atribut `litr` nikoliv na již dříve nachystaný `pinta`.

### Řešení pomocí metody

---

Tvoje první reakce může být doplnění **vhodné metody**.

Tady je situace samozřejmě schůdná, ale musíš **rozbít stávající zápis**.

Resp. každý kdo s takovým skriptem pracuje, bude mít zápis rozházený:

In [8]:
class ObjemovyKonvertor:

    def __init__(self, litr: int):
        self.litr = litr
        self.koeficient_pinta = 1.7598
        
    def pinty(self) -> float:
        return self.litr * self.koeficient_pinta

In [9]:
dva_litry = ObjemovyKonvertor(2)

In [10]:
dva_litry.pinty()

3.5196

<br>

Všude, kde uživatel pracoval **s instancemi a atributy**, bude muset najednou upravovat zápis:

In [11]:
dva_litry.litr = 5

In [12]:
dva_litry.litr

5

In [13]:
dva_litry.pinty()

8.799

<br>

Výsledek je nyní tedy správný.

Ale oprava stávajícího zápisu může být **náročná**. Obzvlášť, pokud třída nebude takhle jednoduchá.

<br>

### Řešení pomocí funkce `property`

---

V jiných jazycích takhle uživatelé sahají ihned po:
1. `setter`,
2. `getter`,
3. `deleter`.

Tedy objekty, které umožní správně zasahovat a nastavovat atributy.

Python v tomto ohledu nabízí několik šidítek, nejprve funkce `property`:

In [14]:
class ObjemovyKonvertor:

    def __init__(self, litr: int):
        self.litr = litr
        self.__koeficient_pinta = 1.7598
        
    @property
    def pinty(self):
        return self.litr * self.__koeficient_pinta

<br>

Velmi zjednoduššeně vytvoříš prakticky metody, ale *interpret* ti k nim dovolí přistoupit jako k atributu.

In [15]:
dva_litry = ObjemovyKonvertor(2)

In [16]:
dva_litry.pinty

3.5196

In [17]:
dva_litry.pinty()

TypeError: 'float' object is not callable

<br>

V aktuálním zápise už **nepoužíváš explicitně metodu a závorky**.

Ale jen přístup přes *atribut* `pinty`.

In [18]:
dva_litry.litr = 3

In [19]:
dva_litry.pinty

5.2794

<br>

Co když ale budeš chtít zadat **přímo pinty** a ty převádět na litry:

In [20]:
dva_litry.pinty = 2

AttributeError: can't set attribute

<br>

*Interpret* tě nenechá zadat výraz pro atribut instance.

<br>

### Dekorátory setter, getter

---

Pokud budeš chtít zadávat **zvlášť litry** a **zvlášť pinty**, potřebuješ svoji třídu správně rozdělit:

In [21]:
class ObjemovyKonvertor:

    def __init__(self):
        self._litr  = 0
        self._pinta = 0
        self.__koeficient_pinta = 1.7598
        
    @property
    def litr(self):
        print("GETTER: odebírám hodnotu 'litr'.")
        return self._litr
    
    @property
    def pinta(self):
        print("GETTER: odebírám hodnotu 'pinta'.")
        return self._pinta

<br>

Pomocí tzv. *getterů* nachystáš atributy `pinta` a `litr`, které budeš nyní použít pokaždé, když atribut vypíšeš:

In [22]:
prevodnik_1 = ObjemovyKonvertor()

In [23]:
print(
    prevodnik_1.litr,
    prevodnik_1.pinta,
    sep="\n"
)

GETTER: odebírám hodnotu 'litr'.
GETTER: odebírám hodnotu 'pinta'.
0
0


<br>

**Gettery** slouží jenom k dotazování a je potřeba je označit jako chráněné proměnné (*weak private*): 

In [24]:
prevodnik_1.litr = 2

AttributeError: can't set attribute

Teď, ale nemáš připravený způsob, jak hodnoty nastavovat.

Nyní můžeš hodnoty **pouze odebírat**.

<br>

Ať můžeš pohodlně měnit hodnoty atributů, potřebuješ nastavit dekorátor `setter`:

In [25]:
class ObjemovyKonvertor:

    def __init__(self):
        self._litr = 0
        self._pinta = 0
        self.__koeficient_pinta = 1.7598

    @property
    def litr(self):
        print("GETTER: odebírám hodnotu 'litr'.")   # POUZE PRO ZNAZORNENI
        return self._litr

    @property
    def pinta(self):
        print("GETTER: odebírám hodnotu 'pinta'.")  # POUZE PRO ZNAZORNENI
        return self._pinta
    
    @litr.setter
    def litr(self, hodnota: float):
        print("SETTER: nastavuji hodnotu 'litr'.")  # POUZE PRO ZNAZORNENI
        self._litr = hodnota  

    @pinta.setter
    def pinta(self, hodnota: float):
        print("SETTER: nastavuji hodnotu 'pinta'.") # POUZE PRO ZNAZORNENI
        self._pinta = hodnota

In [26]:
prevodnik_1 = ObjemovyKonvertor()

In [27]:
print(prevodnik_1.litr)

GETTER: odebírám hodnotu 'litr'.
0


In [28]:
prevodnik_1.litr = 2

SETTER: nastavuji hodnotu 'litr'.


In [29]:
prevodnik_1.pinta = 2

SETTER: nastavuji hodnotu 'pinta'.


In [30]:
print(
    prevodnik_1.litr,
    prevodnik_1.pinta,
    sep='\n'
)

GETTER: odebírám hodnotu 'litr'.
GETTER: odebírám hodnotu 'pinta'.
2
2


<br>

V tomto případě Python nejprve spustí `prevodnik_1.litr` a `prevodnik_1.pinta`. Proto vidíš:
```
GETTER: odebírám hodnotu 'litr'.
GETTER: odebírám hodnotu 'pinta'.
```

Teprve později, samotná funkce `print` vypisuje výstupy:
```
2
2
```

<br>

Teď když můžeš elegantně zadávat jak pinty, tak litry, můžeš doplnit metody pro přepočty:

In [33]:
class ObjemovyKonvertor:

    def __init__(self):
        self._litr = 0
        self._pinta = 0
        self.__koeficient_pinta = 1.7598
        
    @property
    def pinta(self):
        print("GETTER: odebírám hodnotu 'pinta'.")
        return self._pinta
    
    @property
    def litr(self):
        print("GETTER: odebírám hodnotu 'litr'.")
        return self._litr
    
    @pinta.setter
    def pinta(self, hodnota: float):
        print("SETTER: nastavuji hodnotu 'pinta'.")
        self._pinta = hodnota
        
    @litr.setter
    def litr(self, hodnota: float):
        print("SETTER: nastavuji hodnotu 'litr'.")
        self._litr = hodnota
        
    def na_pinty(self):
        if self._litr:
            return self.__koeficient_pinta * self._litr   # round()
        else:
            print("Nenastavená hodnota pro 'litr'")
    
    def na_litry(self):
        if self._pinta:
            return self._pinta / self.__koeficient_pinta  # round()
        else:
            print("Nenastavená hodnota pro 'pinta'")

In [34]:
prevodnik_1 = ObjemovyKonvertor()

In [35]:
prevodnik_1.na_pinty()

Nenastavená hodnota pro 'litr'


In [36]:
prevodnik_1.litr

GETTER: odebírám hodnotu 'litr'.


0

In [37]:
prevodnik_1.litr = 3

SETTER: nastavuji hodnotu 'litr'.


In [38]:
prevodnik_1.litr

GETTER: odebírám hodnotu 'litr'.


3

In [39]:
prevodnik_1.na_pinty()

5.2794

In [40]:
prevodnik_1.pinta = 3

SETTER: nastavuji hodnotu 'pinta'.


In [41]:
prevodnik_1.na_litry()

1.7047391749062393

<br>

Je poměrně běžné, že v rámci `setteru` uvádíš ověřovací podmínky:

In [51]:
class ObjemovyKonvertor:

    def __init__(self, litr: int = 0, pinta: int = 0):
        self._litr = 0
        self._pinta = 0
        self.__koeficient_pinta = 1.7598
        
    @property
    def pinta(self):
        print("GETTER: odebírám hodnotu 'pinta'.")
        return self.pinta
    
    @property
    def litr(self):
        print("GETTER: odebírám hodnotu 'litr'.")
        return self.litr
    
    @pinta.setter
    def pinta(self, hodnota: int):
        print("SETTER: nastavuji hodnotu 'pinta'.")
        
        if not isinstance(hodnota, int):
            print("Špatný datový typ pro 'pinta'")
        else:
            self.pinta = hodnota
        
    @litr.setter
    def litr(self, hodnota: int):
        print("SETTER: nastavuji hodnotu 'litr'.")
        
        if not isinstance(hodnota, int):
            print("Špatný datový typ pro 'litr'")
        else:
            self.litr = hodnota
        
#     def na_pinty(self):
#         if self._litr > 0:
#             return self.__koeficient_pinta * self._litr
#         else:
#             print("Nenastavená hodnota pro 'litr'")
    
#     def na_litry(self):
#         if self._pinta > 0:
#             return self._pinta / self.__koeficient_pinta 
#         else:
#             print("Nenastavená hodnota pro 'pinta'")

In [53]:
# prevodnik_1 = ObjemovyKonvertor()

In [44]:
prevodnik_1.litr = "5"

SETTER: nastavuji hodnotu 'litr'.
Špatný datový typ pro 'litr'


In [45]:
prevodnik_1.litr

0

In [46]:
prevodnik_1.litr = 5

SETTER: nastavuji hodnotu 'litr'.


In [47]:
prevodnik_1.na_pinty()

8.799

<br>

Podobným způsobem můžeš definovat také dekorátor `deleter`, který hodnotu odstraní:
```python
    @litr.deleter
    def litr(self):
        del self._litr

```

In [48]:
a = {'a': 1, 'b': 2}

In [49]:
del a['a']

In [50]:
a

{'b': 2}

In [ ]:
# note. Myslet na ukázku RecursionError (nesprávné instancování s kolizí jmen atributu a metody)

<br>

### 🧠 CVIČENÍ 🧠, Vytvoř třídu `Plocha`, která bude tvořena následovným:
---

1. Definuj třídu `Plocha`,
2. vytvoř *instanční konstruktor*, který nepotřebuje parametry,
3. *instanční konstruktor* udává pouze *weak private* atribut `self._prumer_cm` nastavený na nulu,
4. vytvoř metodu `getter` (pomocí dekorátoru) se jménem `prumer_cm`,
5. vytvoř metodu `setter` (pomocí dekorátoru), která ověří, že zadaný parametr `hodnota` je buď `int` nebo `float`, jinak vypíše `"Špatný datový typ pro 'prumer_cm'"`
6. vytvoř *instanční metodu* `vypocitej_plochu_kruhu`, která pomocí konstanty `math.pi` (z knihovny `math`) vrátí plochu kruhu,
7. vytvoř statickou metodu `vypocitej_obvod_kruhu`, která za pomoci parametru `polomer` vrátí obvod kružnice,
8. vytvoř magickou metodu `__str__`, která naformátuje výstup do stringu: `Kruh s průměrem: <polomer>, plochou: <plocha>.`

In [66]:
import math

In [56]:
isinstance("1", (int, float))

False

In [71]:
class Plocha:
    
    def __init__(self):
        self._prumer_cm = ""
    
    @property
    def prumer_cm(self):
        return self._prumer_cm
    
    @prumer_cm.setter
    def prumer_cm(self, hodnota):
        if isinstance(hodnota, int) or isinstance(hodnota, float):  # (int, float, decimal.Decimal)
            self._prumer_cm = hodnota
        else:
            # logging.warning()
            raise TypeError("Špatný datový typ pro 'prumer_cm'")
    
    def vypocitej_plochu_kruhu(self):
        polomer = self._prumer_cm / 2
        return math.pi * ((polomer)**2)

    def vypocitej_obvod_kruhu(self, polomer):
        return 2* math.pi * polomer
    
    def __str__(self):
        return f"Kruh s průměrem: {self._prumer_cm}, plochou: {round(self.vypocitej_plochu_kruhu(), 3)}."

In [72]:
p1 = Plocha()

In [73]:
p1.prumer_cm = 10

In [69]:
p1.prumer_cm

10

In [61]:
p1.prumer_cm = "10"

TypeError: Špatný datový typ pro 'prumer_cm'

In [74]:
print(
    p1,
    str(p1),
    sep='\n'
)

Kruh s průměrem: 5.0, plochou: 78.54.
Kruh s průměrem: 5.0, plochou: 78.54.


<details>
    <summary>▶️ Řešení</summary>
    
```python
class Plocha:
    def __init__(self):
        self._prumer_cm = 0.0

    @property
    def prumer_cm(self):
        return self._prumer_cm

    @prumer_cm.setter
    def prumer_cm(self, hodnota: float):
        if not isinstance(hodnota, (float, int)):
            print("Atribut 'hodnota' není číselný datový typ.")
        else:
            self._prumer_cm = hodnota

    def vypocitej_plochu_kruhu(self) -> float:
        return math.pi * (self._prumer_cm ** 2)

    @staticmethod
    def vypocitej_obvod_kruhu(polomer) -> float:
        return 2 * math.pi * polomer

    def __str__(self) -> str:
        return f"Kruh s průměrem: {self._prumer_cm}, plochou: {self.vypocitej_plochu_kruhu()}."
```
</details>

<br>

### 🧠 CVIČENÍ 🧠, Vytvoř třídu `TeplotniPrevodnik`, která bude tvořena následovným:
---

1. **Definuj novou třídu** `TeplotniPrevodnik`,
2. definuj instanční konstruktor `__init__`, který vytvoří **jeden instanční atribut**: `self._teplota_celsia` (defaultně 0.0),
3. definuj `@property` se jménem `teplota_celsia` s příslušným *getterem*, který vrátí hodnotu `self._teplota_celsia`,
4. definuj `@property` se jménem `teplota_celsia` s příslušným *setterem*, který provede kontrolu, že hodnota není nižší než **-273.15 stupňů Celsia** (jinak `ValueError`),
5. definuj `@property` se jménem `teplota_fahrenheit` s příslušným *getterem*, který vrátí hodnotu `self._teplota_fahrenheit`,
6. definuj `@property` se jménem `teplota_fahrenheit` s příslušným *setterem*,
7. definuj **statickou metodu** `celsia_na_fahrenheit`, která přijme teplotu **v stupních Celsia** a vrátí odpovídající teplotu **ve stupních Fahrenheit**,
8. definuj **statickou metodu** `fahrenheit_na_celsia`, která přijme teplotu **ve stupních Fahrenheit** a vrátí odpovídající teplotu **v stupních Celsia**.

In [76]:
class TeplotniPrevodnik:
    def __init__(self):
        self._teplota_celsia = 0.0

    @property
    def teplota_celsia(self):
        return self._teplota_celsia

    @teplota_celsia.setter
    def teplota_celsia(self, value):
        # logger.info("Prevadni na stupne Celsia..")
        if value < -273.15:
            # logger.warning("...")
            raise ValueError("Teplota nemůže být nižší než -273.15 stupňů Celsia.")
        self._teplota_celsia = value
        # logger.info("Hodnota ulozena jako 'teplota_celsia'")

    @property
    def teplota_fahrenheit(self):
        return TeplotniPrevodnik.celsia_na_fahrenheit(self._teplota_celsia)  # 

    @teplota_fahrenheit.setter
    def teplota_fahrenheit(self, value):
        self._teplota_celsia = TeplotniPrevodnik.fahrenheit_na_celsia(value) # 

    @staticmethod
    def celsia_na_fahrenheit(teplota_celsia):
        return teplota_celsia * 9 / 5 + 32

    @staticmethod
    def fahrenheit_na_celsia(teplota_fahrenheit):
        return (teplota_fahrenheit - 32) * 5 / 9

    def __str__(self):
        return f"Teplota: {self._teplota_celsia}°C, {self.teplota_fahrenheit}°F"

In [77]:
prevodnik = TeplotniPrevodnik()

In [78]:
prevodnik.teplota_celsia

0.0

In [82]:
prevodnik.teplota_celsia = 25

In [83]:
prevodnik.teplota_celsia

25

In [84]:
prevodnik.teplota_fahrenheit

77.0

<details>
    <summary>▶️ Řešení</summary>
    
```python
class TeplotniPrevodnik:
    def __init__(self, teplota: float = 0.0) -> None:
        """
        Vytvoří novou instanci třídy, s defaultní hodnotou
        teploty ve stupních Celsia.
        """
        self._teplota_celsia = teplota

    @property
    def teplota_celsia(self):
        return self._teplota_celsia

    @teplota_celsia.setter
    def teplota_celsia(self, hodnota: float):
        if hodnota < -273.15:
            raise ValueError("Teplota nemůže být nižší než absolutní nula!")
        self._teplota_celsia = hodnota

    @property
    def teplota_fahrenheit(self):
        return TeplotniPrevodnik.celsia_na_fahrenheit(self._teplota_celsia)

    @teplota_fahrenheit.setter
    def teplota_fahrenheit(self, hodnota):
        self._teplota_celsia = TeplotniPrevodnik.fahrenheit_na_celsia(hodnota)

    @staticmethod
    def celsia_na_fahrenheit(celsia):
        return celsia * 9 / 5 + 32

    @staticmethod
    def fahrenheit_na_celsia(fahrenheit):
        return (fahrenheit - 32) * 5 / 9
```
</details>

<img src="pillars.png" width="900" style="margin-left:auto; margin-right:auto"/>

OOP je *paradigma*, které používá "objekty" – struktury, návody obsahující:
1. **Atributy**, tedy *argumenty* (popisují, jaký objekt je, jak vypadá),
2. **vlastnosti**, tedy *metody* (popisují, co objekt umí).

**Efektivní využití OOP** může vést k software, který je snadněji pochopitelný, flexibilnější a údržba, což přináší řadu výhod při vývoji složitých aplikací.

Proto je ale nutné vědět, na čem všem toto *paradigma* stojí.

<br>

Mezi **4 základní koncepty OOP** je patří:
1. **Polymorfismus**,
2. **zapouzdření**,
3. **dědičnost**,
4. **abstrakce**.

## Polymorfismus ( ~polymorphism)

---

<img href="https://i.imgur.com/COUOylr.png" width="1000" style="margin-left:auto; margin-right:auto">

### Terminologie

---

**Polymorfismus** znamená *mnohotvárnost*.

Lidsky si pod tímto můžeš představit situaci, kdy lze jednu věc popsat různými aplikacemi.

Například **pozdrav**. Slušný pozdrav můžeš vyjádřit několika způsoby, nebo dokonce různými jazyky.

<br>

*Polymorfismus* přispívá k flexibilitě a rozšířitelnosti kódu tím, že umožňuje programátorům používat **objekty různých tříd zaměnitelně**.

### Význam v Pythonu

---

Ve světě programování  to zase znamená, že jeden objekt, může plnit různé účely:

In [85]:
print(type(1))

<class 'int'>


In [87]:
a = 1

In [89]:
print(a.__add__(9))

10


In [91]:
print(
    1 + 1,            # operátor '+' s celočíselnými operandy,
    "Lekce" + " #03", # operátor '+' se stringovými operandy,
    sep="\n"
)

2
Lekce #03


<br>

Tento koncept už znáš.

Operátor `+` totiž znamená **pro dva odlišné datové typy, dvě odlišné procedůry**:
1. `int`, sčítání,
2. `str`, spojování (~concatenation).

<br>

Takovou mnohotvárnost, ale neznáš jen mezi `int` a `str`.

Podobného chování si můžeš všimnout i pro uživ. funkci `len`:

In [92]:
print(
    len("Matous"),                                                             # str
    len(["město", "moře", "kuře", "stavení"]),                                 # list
    len({"jmeno": "Matous", "prijmeni": "Holinka", "email": "matous@nic.cz"}), # dict
    sep="\n"
)

6
4
3


In [ ]:
"Matous".__len__()

<br>

Vidíš, že funkce `len` umí pracovat s různými datovými typy.

<br>

Vrací specifickou hodnotu pro každý datový typ zvlášť. Ale pořád používáš jednu a tutéž funkci.
1. `str`, délka řetězce,
2. `list`, počet údajů v sekvenci,
3. `dict`, počet klíčů v objektu

<br>

### Využití v OOP

---

Dále tento koncept umožňuje přepisovat jména a použití **metod**:

In [93]:
class CsvProcessor:
    def precti_soubor(self) -> str:
        return "Otevírám CSV soubor..."

In [94]:
class JsonProcessor:
    def precti_soubor(self) -> str:
        return "Otevírám JSON soubor..."

In [98]:
csv_soubor = CsvProcessor()
json_soubor = JsonProcessor()

In [99]:
for soubor in (csv_soubor, json_soubor):
    print(soubor.precti_soubor())

Otevírám CSV soubor...
Otevírám JSON soubor...


<br>

Prakticky to tedy znamená, že *polymorfismus* umožní objektům **různých tříd**, pracovat **se stejnojmennými metodami**.

S dalším využitím *polymorfismu* se můžeš setkat u dědičnosti (později v materiálech).

<br>

### 🧠 CVIČENÍ 7 🧠, Vytvoř třídy `CsvProcessor` a `TxtProcessor`:
---

- Definuj třídu `CsvProcessor`, s metodou `__init__`, která potřebuje jen parametr `jmeno_souboru`,
- zadej třídě `CsvProcessor` třídní atribut `ext` s hodnotou `.csv`,
- metoda `__init__` sama ověří, jestli proměnná obsahuje správnou příponu,
- pokud je přípona `.csv`, definuj instanční atribut, jinak vyvolej výjimku s textem: `Špatný formát souboru`,
- vytvoř metodu `nacti_obsah`, která přečte a vrátí obsah **csv** souboru,
- definuj třídu `TxtProcessor`, s metodou `__init__`, která potřebuje jen parametr `jmeno_souboru`,
- zadej třídě `TxtProcessor` třídní atribut `ext` s hodnotou `.txt`,
- metoda `__init__` sama ověří, jestli proměnná obsahuje správnou příponu,
- pokud je přípona `.txt`, definuj instanční atribut, jinak vyvolej výjimku s textem: `Špatný formát souboru`,
- vytvoř metodu `nacti_obsah`, která přečte a vrátí obsah **txt** souboru.

Před spuštěním ukázky si nezapomeň vytvořit pokusné soubory.

In [100]:
from pathlib import Path

In [102]:
Path("/home/matous/t1.txt").suffix

'.txt'

In [103]:
import os

In [105]:
# dir(os.path)

In [116]:
from pathlib import Path
from pandas import read_csv

class CsvProcessor:
    ext: str = ".csv"

    def __init__(self, jmeno_souboru):
        # self.ext = ".csv"
        if Path(jmeno_souboru).suffix == self.ext:  # os.path.
            self.jmeno_souboru = jmeno_souboru
        else:
            raise ValueError("Špatný formát souboru")

    def nacti_obsah(self):
        return read_csv(self.jmeno_souboru, sep=";")

class TxtProcessor:
    def __init__(self, jmeno_souboru):
        self.jmeno_souboru = jmeno_souboru
        self.ext = ".txt"
        if self.jmeno_souboru.endswith(self.ext):
            pass
        else:
            raise ValueError("Špatný formát souboru")

    def nacti_obsah(self):
        with open(self.jmeno_souboru, "r", encoding="utf-8") as f:
            return f.read()

In [ ]:
csv_1 = CsvProcessor("csv_pokus.csv")
txt_1 = TxtProcessor("txt_pokus.txt")

In [107]:
!ls -l | grep "csv"

-rw-rw-r-- 1 matous matous     51 lis 22 20:32 test_csv.csv


In [117]:
csv_2 = CsvProcessor("test_csv.csv")

In [118]:
csv_2.__dict__

{'jmeno_souboru': 'test_csv.csv'}

In [119]:
obsah = csv_2.nacti_obsah()

In [122]:
obsah['jmeno']

0    Matous
1      Adam
Name: jmeno, dtype: object

<details>
    <summary>▶️ Řešení</summary>
    
```python
from pathlib import Path

from pandas import read_csv


class CsvProcessor:
    ext: str = ".csv"

    def __init__(self, jmeno_souboru: str):
        if Path(jmeno_souboru).suffix != self.ext:
            raise Exception("Špatný formát souboru")
        self.jmeno_souboru = jmeno_souboru

    def nacti_obsah(self):
        return read_csv(self.jmeno_souboru)


class TxtProcessor:
    ext: str = ".txt"

    def __init__(self, jmeno_souboru: str):
        if Path(jmeno_souboru).suffix != self.ext:
            raise Exception("Špatný formát souboru")
        self.jmeno_souboru = jmeno_souboru

    def nacti_obsah(self):
        with open(self.jmeno_souboru) as txt:
            return txt.read()
```
</details>

<br>

## Zapouzdření (~encapsulation)

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.uDpIgfGQDzGDPvxtzsz-5AHaHa%26pid%3DApi&f=1&ipt=eb0bca94ab317533d3313fe92c84884bff2b45c5ce18cfdb8341427e8a8311fe&ipo=images" width="300" style="margin-left:auto; margin-right:auto">

### Terminologie

---

Tento pojem obecně označuje **nějaké skrývání**.

Jde především o práci **s privátními a chráněnými** atributy nebo metodami.

Účelem tohoto skrývání, nebo zapouzdření je ochrana.

<br>

Aby programátoři nepřepisovali, co nemají a tím nezpůsobili **komplikace** zdrojového kódu.

### Bez konceptu zapouzdření

---

V kombinaci s **chráněnými objekty** chceš některé procesy zajistit a současně nekomplikovat uživatelské použití:

In [123]:
class MojeAplikace:
    
    def __init__(self, jmeno: str, kredit: int):
        self.jmeno = jmeno
        self.kredit = kredit
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.kredit}"

In [124]:
uzivatel_matous = MojeAplikace("Matouš", 100)

In [125]:
print(uzivatel_matous.vypis_status())

Uživatel: Matouš, dostupné kredity: 100


<br>

V aktuálním stavu třídy může **kdokoliv přistup z venku** naší třídy `MojeAplikace` a nedopatřením přepsat hodnoty:

In [126]:
uzivatel_matous.kredit = 10_000

In [127]:
print(uzivatel_matous.vypis_status())

Uživatel: Matouš, dostupné kredity: 10000


In [128]:
uzivatel_matous.kredit = "10_000"

In [129]:
print(uzivatel_matous.vypis_status())

Uživatel: Matouš, dostupné kredity: 10_000


In [130]:
uzivatel_matous.kredit += 5_000

TypeError: can only concatenate str (not "int") to str

<br>

### Privátní objekty

---

<br>

Proto je vhodné, zápis doplnit:
1. **chráněnými** `__jmeno`,
2. **privátními** `_jmeno`.

In [131]:
class MojeAplikace:
    
    def __init__(self, jmeno: str, kredit: int):
        self.jmeno = jmeno
        self.__kredit = kredit
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.__kredit}"

In [132]:
uzivatel_lukas = MojeAplikace("Lukáš", 1000)

In [133]:
print(uzivatel_lukas.vypis_status())

Uživatel: Lukáš, dostupné kredity: 1000


In [134]:
uzivatel_lukas.kredit = 1_000_000

In [135]:
print(uzivatel_lukas.vypis_status())

Uživatel: Lukáš, dostupné kredity: 1000


In [136]:
uzivatel_lukas.__dict__

{'jmeno': 'Lukáš', '_MojeAplikace__kredit': 1000, 'kredit': 1000000}

<br>

Koncept, na kterém je *privátní objekt* v Pythonu postavený, se označuje *name mangling* (tedy *komolení* jmen objektů).

Ale takové opatření pořád neřeší problém, pokud instance dostane třeba **nevhodný datový typ**:

In [ ]:
uzivatel_lukas._MojeAplikace__kredit = "1_000_000"

In [ ]:
print(uzivatel_lukas.vypis_status())

<br>

### Gettery, settery

---

<br>

Pro úplný pořádek je ještě nutné stanovit formu, kterou bude atribut zadáván:

In [137]:
class MojeAplikace:
    
    def __init__(self, jmeno: str):
        self.jmeno = jmeno
        self.__kredit = 0
        
    @property
    def kredit(self):
        """Getter metoda"""
        return self.__kredit
    
    @kredit.setter
    def kredit(self, hodnota: int) -> None:
        """Setter metoda"""
        if isinstance(hodnota, int):
            self.__kredit = hodnota
        else:
            raise Exception("Zadaná hodnota není celé číslo")
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.__kredit}"

In [138]:
uzivatel_lukas = MojeAplikace("Lukáš")

In [139]:
uzivatel_lukas.kredit = 1000

In [140]:
print(uzivatel_lukas.vypis_status())

Uživatel: Lukáš, dostupné kredity: 1000


<br>

Nyní není možné zadat jiný datový typ, než celé číslo:

In [141]:
uzivatel_lukas.kredit = "1000"

Exception: Zadaná hodnota není celé číslo

<br>

Pomocí defaultní výjimky `Exception` nyní uživatel vidí, že není možné pracovat s jiným datovým typem.

#### Souhrn k zapouzdření
* **bezpečnost**, u privátních objektů je navíc vrstva ochrany proti nechtěnému přepsaní, nebo použití nesprávného datového typu,
* **kontrola**, ostatní programátoři musí dodržovat postupy, pro manipulaci s privátními objekty,
* **jednoduchost**, rozdělování zadávání a získání hodnot jsou oddělené objekty,
* **čitelnost**, spojování objektů k příslušným třídám dělá zápis přehlednější.

<br>

### 🧠 CVIČENÍ 8 🧠, Vytvoř třídu `AutomobilTesla` a doplň následující:

- Definuj třídu `AutomobilTesla`, s metodou `__init__`, která potřebuje jen parametr `uzivatel` a `cislo`,
- vytvoř **getter a setter** metody pro atribut `max_rychlost`,
- omez zadávání `max_rychlost` jenom pro datový typ `int` z intervalu `0 - 320`,
- pokud uživatel nedodrží podmínky, vyvolej výjimku s textem: `Nepřijatelný datový typ nebo hodnota (< 320 km/h)`,
- vytvoř **getter a setter** metody pro atribut `max_dojezd`,
- omez zadávání `max_dojezd` jenom pro datový typ `int` z intervalu `0 - 590`,
- pokud uživatel nedodrží podmínky, vyvolej výjimku s textem: `Nepřijatelný datový typ nebo hodnota (< 600 km)`,
- přepiš magickou metodu `__str__`, aby formátovala tento výstup: `Uživatel: <uzivatel>, sériové číslo: <cislo>`.

In [ ]:
tesla_model_s = AutomobilTesla("Matouš", "1234ABCE5678")

In [ ]:
tesla_model_s.__dict__

In [ ]:
tesla_model_s.max_rychlost = 250

In [ ]:
tesla_model_s.max_rychlost

In [ ]:
tesla_model_s.__dict__

<details>
    <summary>▶️ Řešení</summary>
    
```python
class AutomobilTesla:
    def __init__(self, uzivatel: str, cislo: int):
        self.uzivatel = uzivatel
        self.cislo = cislo

    @property
    def max_rychlost(self):
        return self.__max_rychlost

    @max_rychlost.setter
    def max_rychlost(self, hodnota: int):
        if isinstance(hodnota, int) and 0 < hodnota < 320:
            self.__max_rychlost = hodnota
        else:
            raise Exception("Nepřijatelný datový typ nebo hodnota (< 320 km/h)")

    @property
    def max_dojezd(self):
        return self.__max_dojezd

    @max_dojezd.setter
    def max_dojezd(self, hodnota):
        if isinstance(hodnota, int) and 0 < hodnota < 590:
            self.__max_dojezd = hodnota
        else:
            raise Exception("Nepřijatelný datový typ nebo hodnota (< 600 km)")


    def __str__(self):
        return f"Uživatel: {self.uzivatel}, sériové číslo: {self.cislo}"
```
</details>

[Formulář po třetí lekci](https://forms.gle/6kLQ1mEpXXHZUbgB8)

---